In [1]:
import pandas as pd
import copy
import time
import numpy as np

## School PC

In [3]:
filepath = r'C:\Users\n10405992\OneDrive - Queensland University of Technology\Raghav Malhotra - RM\Papers\RCS\RCS Codes\Path Choice\Data\final\network_13_07_2021.csv'
linear_distance = pd.read_csv(r'C:\Users\n10405992\OneDrive - Queensland University of Technology\Raghav Malhotra - RM\Papers\RCS\RCS Codes\Path Choice\Data\final\Final linear network.csv').set_index("ID")
shortest_distance = pd.read_csv(r"C:\Users\n10405992\OneDrive - Queensland University of Technology\Raghav Malhotra - RM\Papers\RCS\RCS Codes\Path Choice\Data\final\distance_matrix_29062021.csv")
network = pd.read_csv(filepath, usecols = ['From_origi', 'To_destina', 'Total_Minu','Total_Kilo'])
network['Total_Minu'] = round(network['Total_Minu'],2)
network['Total_Kilo'] = round(network['Total_Kilo'],4)

In [4]:
net = pd.read_csv(filepath)
net

,RouteName,StartTime,EndTime,StartTimeU,EndTimeUTC,Total_Mile,Total_Kilo,Total_Minu,TotalWait_,TotalViola,...,From_id,From_origi,From_x,From_y,From_link,To_id,To_destina,To_x,To_y,Shape_Leng
0,9.991971e+10,NaN,NaN,NaN,NaN,7.599278,12.229877,8.930338,0,0,...,1,999197,152.920984,-27.599225,9.991971e+10,1,10940,153.014630,-27.638840,13796.240340
1,1.213801e+09,NaN,NaN,NaN,NaN,6.913524,11.126261,10.639143,0,0,...,10,121380,153.279680,-27.580500,1.213801e+09,10,1075,153.215750,-27.522430,12551.809600
2,1.107117e+07,NaN,NaN,NaN,NaN,3.141876,5.056370,5.964268,0,0,...,100,1107,153.149840,-27.509280,1.107117e+07,100,1168,153.193660,-27.525300,5699.975340
3,1.077130e+07,NaN,NaN,NaN,NaN,3.126653,5.031870,6.167305,0,0,...,101,1077,153.235670,-27.524080,1.077130e+07,101,1298,153.230670,-27.488890,5673.919147
4,1.733122e+10,NaN,NaN,NaN,NaN,3.126350,5.031383,3.807592,0,0,...,102,173312,153.033590,-27.491630,1.733122e+10,102,18039,153.032650,-27.447060,5692.316530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3908,1.805910e+09,NaN,NaN,NaN,NaN,0.006752,0.010867,0.017706,0,0,...,3909,18059,153.029560,-27.446230,1.805910e+09,3909,10083,153.029670,-27.446350,12.299559
3909,1.799991e+10,NaN,NaN,NaN,NaN,0.003478,0.005597,0.007473,0,0,...,3910,179999,153.011862,-27.435412,1.799991e+10,3910,10263,153.011915,-27.435415,6.290211
3910,1.089111e+09,NaN,NaN,NaN,NaN,0.002803,0.004511,0.006436,0,0,...,3911,10891,153.013469,-27.471728,1.089111e+09,3911,10846,153.013651,-27.471686,5.269600
3911,1.215091e+10,NaN,NaN,NaN,NaN,0.001714,0.002759,0.007715,0,0,...,3912,121509,153.025635,-27.474614,1.215091e+10,3912,10360,153.025320,-27.474078,3.113218


## Home PC

In [3]:
tt = network.groupby('From_origi')[['To_destina', 'Total_Minu']].apply(lambda x: x.set_index('To_destina').to_dict(orient='index')).to_dict()
for key in tt:
        for x in tt[key]:
            for y in tt[key][x]:
                tt[key][x] = tt[key][x][y]

In [5]:
dist_matrix = network.groupby('From_origi')[['To_destina', 'Total_Kilo']].apply(lambda x: x.set_index('To_destina').to_dict(orient='index')).to_dict()
for key in dist_matrix:
        for x in dist_matrix[key]:
            for y in dist_matrix[key][x]:
                dist_matrix[key][x] = dist_matrix[key][x][y]*1000

In [6]:
linear_distance = linear_distance.to_dict()

In [7]:
shortest_dist = shortest_distance.groupby('OriginOID')[['DestinationOID', 'Total_Distance']].apply(lambda x: x.set_index('DestinationOID').to_dict(orient='index')).to_dict()
for key in shortest_dist:
        for x in shortest_dist[key]:
            for y in shortest_dist[key][x]:
                shortest_dist[key][x] = shortest_dist[key][x][y]*1000

In [8]:
def get_path_weight(path, net):
    weight = 0
    for i in range(len(path)-1):
        weight += net[path[i]][path[i+1]]
    return weight

In [9]:
def temporal_constraint(path_segment, main_path, factor):
    min_time = 9999999
    for path in main_path:
        time = get_path_weight(path,tt)
        if time<min_time:
            min_time = time
    path_time = get_path_weight(path_segment,tt)
    if path_time < (factor* min_time):
        return True
    else:
        return False

In [10]:
def similarity_constraint(path_segment, main_paths, factor):
    main_path_links = []
    for path in main_paths:
        z = []
        for i in range(len(path)-1):
            z.append(int(str(path[i]) + str(path[i+1])))
        main_path_links.append(z)
    path_links = []
    for i in range(len(path_segment)-1):
        path_links.append(int(str(path_segment[i]) + str(path_segment[i+1])))
    
    for i in range(len(main_path_links)):
        common_length = 0
        total_length = get_path_weight(main_paths[i], dist_matrix)
        for j in range(len(path_links)):
            if path_links[j] in main_path_links[i]:
                common_length += dist_matrix[path_segment[j]][path_segment[j+1]]
        if common_length/total_length > factor:
            return False
        else:
            continue
    return True
        

In [11]:
def loop_constraint(path_segment, factor):
    enum_segments = []
    constraint = True
    total_w = 0
    for i in range(len(path_segment)+1):
        for j in range(3,len(path_segment)+1):
            if len(path_segment[i:j])>2:
                segment = path_segment[i:j]
                start = time.time()
                path_len = get_path_weight(segment, dist_matrix)
                total_w+= 1
                if path_len > factor * shortest_dist[segment[0]][segment[-1]]:
                    constraint = False
                    break
        if constraint:
            continue
        else:
            break
    return constraint

In [30]:
def create_new_level(x, start, end, factor):
    y = []
    for segment in x:
        if segment[-1] == end:
            continue
        new = copy.deepcopy(segment)
        try:
            child = list(tt[segment[-1]])
        except:
            continue
        for c in child:
            try:
                if c in segment:
                    continue
                if linear_distance[str(c)][end] > 0.05 * linear_distance[str(segment[0])][end]:
                    if linear_distance[str(c)][end]>factor*linear_distance[str(segment[-1])][end]  or shortest_dist[c][end] > 1.5*shortest_dist[start][end]:
                       
                        continue
            except:
                print("error")
                print(c, segment)
            new.append(c)
            y.append(new)
            new = copy.deepcopy(segment)
    return y

In [17]:
def check_last(list_, val):
    counter = 0
    for i in range(len(list_)):
        if list_[i][-1] == val :
            counter += 1
    if counter == (len(list_)):
        return True
    else:
        return False

In [18]:
def new_loop_constraint(path_segment, factor):
    constraint = True
    path_weight = get_path_weight(path_segment, dist_matrix)
    if path_weight>factor*shortest_dist[path_segment[0]][path_segment[-1]]:
        constraint = False
        return constraint
    half_segments = divide_segment(path_segment, 2)
    for path in half_segments:
        if path_weight>factor*shortest_dist[path_segment[0]][path_segment[-1]]:
            constraint = False
            return constraint
    fourth_segments = divide_segment(path_segment, 4)
    paths_check = []
    paths_check.append(fourth_segments[1].extend(fourth_segments[2]))
    for path in fourth_segments:
        paths_check.append(path)
    for path in paths_check:
         if path_weight>factor*shortest_dist[path_segment[0]][path_segment[-1]]:
            constraint = False
            return constraint
    return constraint

In [31]:
def bnb(graph, start, end, constraints):
    track_incoming_paths = {}
    for node in graph:
        track_incoming_paths[node] = []
    for node in linear_distance:
        if node not in track_incoming_paths:
            track_incoming_paths[node] = []
    current_level = [[start]]
    next_level = create_new_level(current_level,start, end, constraints[0])
    boo = False
    while not boo:
        boo = check_last(next_level, end)
        for path_segment in next_level:
            if temporal_constraint(path_segment, track_incoming_paths[path_segment[-1]], constraints[1]):
                if similarity_constraint(path_segment, track_incoming_paths[path_segment[-1]], constraints[3]):
                    loop= loop_constraint(path_segment, constraints[2])
                    if loop:
                        track_incoming_paths[path_segment[-1]].append(path_segment)
                    else:
                        next_level.remove(path_segment)
                else:
                    next_level.remove(path_segment)
            else:
                next_level.remove(path_segment)
        current_level = []
        for path_segment in next_level:
            current_level.append(path_segment)
        next_level = []
        next_level = create_new_level(current_level, start,end, constraints[0])
   
    return track_incoming_paths[end]